# **Auto fit the arima model**

In [ ]:
!pip install auto_arima

ERROR: Could not find a version that satisfies the requirement auto_arima (from versions: none)
ERROR: No matching distribution found for auto_arima


In [ ]:
!pip install numpy==1.24.4
!pip install pmdarima==1.8.5


  Using cached pmdarima-1.8.5.tar.gz (639 kB)
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for pmdarima
  Running setup.py clean for pmdarima
Failed to build pmdarima
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (pmdarima)


In [ ]:
import pandas as pd
from pmdarima import auto_arima
import matplotlib.pyplot as plt

# Load and prepare data
df = pd.read_csv("bengaluru_potato.csv")
df['date'] = pd.to_datetime(df['date'])
df = df.sort_values(by='date')
retail_series = df['bengaluru_retailprice']

# Auto ARIMA model
auto_model = auto_arima(retail_series, seasonal=False, trace=True,
                        error_action='ignore', suppress_warnings=True, stepwise=True)

# Summary
print(auto_model.summary())

# Forecast next 12 months
forecast = auto_model.predict(n_periods=12)

# Generate future dates
future_dates = pd.date_range(start=df['date'].iloc[-1] + pd.DateOffset(months=1), periods=12, freq='MS')

# Plot
plt.figure(figsize=(10, 5))
plt.plot(df['date'], retail_series, label='Original')
plt.plot(future_dates, forecast, label='Forecast', color='green', marker='o')
plt.legend()
plt.title('Auto ARIMA Forecast – Next 12 Months')
plt.xlabel('Date')
plt.ylabel('Retail Price')
plt.grid(True)
plt.show()


ModuleNotFoundError: No module named 'pmdarima'

In [ ]:
auto_model.summary()

# **Evaluate model predictions**

In [ ]:
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

# Load and prepare data
df = pd.read_csv("bengaluru_potato.csv")
df['date'] = pd.to_datetime(df['date'])
df = df.sort_values(by='date')
retail_series = df['bengaluru_retailprice']

# Split into train (90%) and test (10%)
split_index = int(len(retail_series) * 0.9)
train = retail_series[:split_index]
test = retail_series[split_index:]

# Fit ARIMA on training data
model = ARIMA(train, order=(1, 1, 1))
fitted_model = model.fit()

# Forecast for test period
forecast = fitted_model.forecast(steps=len(test))

# Evaluation metrics
mae = mean_absolute_error(test, forecast)
rmse = np.sqrt(mean_squared_error(test, forecast))
mape = np.mean(np.abs((test - forecast) / test)) * 100

print(f"MAE  = {mae:.2f}")
print(f"RMSE = {rmse:.2f}")
print(f"MAPE = {mape:.2f}%")


In [ ]:
import matplotlib.pyplot as plt

# Plot actual vs predicted
plt.plot(test.index, test, label='Actual', marker='o')
plt.plot(test.index, forecast, label='Predicted', marker='o', color='green')
plt.title('ARIMA: Actual vs Predicted (Test Set)')
plt.xlabel('Index')
plt.ylabel('Retail Price')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
!pip install scikit-fuzzy
!pip install scikit-learn


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import skfuzzy as fuzz
import matplotlib.pyplot as plt

# Load and prepare data
df = pd.read_csv("bengaluru_potato.csv")
df['date'] = pd.to_datetime(df['date'])
df = df.sort_values('date')

# Select relevant features (you can add more features)
features = df[['bengaluru_wholesaleprice', 'bengaluru_volume', 'bengaluru_avg._temperature']]
target = df['bengaluru_retailprice'].values

# Normalize features
scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(features)

# Apply Fuzzy C-Means clustering
n_clusters = 3
cntr, u, _, _, _, _, _ = fuzz.cluster.cmeans(
    scaled_features.T, c=n_clusters, m=2.0, error=0.005, maxiter=1000, init=None)

# Assign each sample to the cluster with highest membership
cluster_membership = np.argmax(u, axis=0)

# Calculate average retail price per cluster (for prediction)
cluster_retail_means = [target[cluster_membership == i].mean() for i in range(n_clusters)]

# Predict retail price based on highest membership cluster
fcm_predictions = [cluster_retail_means[i] for i in cluster_membership]

# Plot actual vs predicted
plt.figure(figsize=(10, 5))
plt.plot(df['date'], target, label='Actual', marker='o')
plt.plot(df['date'], fcm_predictions, label='Predicted (Fuzzy C-Means)', marker='x')
plt.title('Fuzzy C-Means: Retail Price Prediction')
plt.xlabel('Date')
plt.ylabel('Retail Price')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
# Store actual and both model predictions
comparison_df = pd.DataFrame({
    'date': df['date'],
    'actual': target,
    'fcm_predicted': fcm_predictions,
    'arima_predicted': [np.nan]*(len(df)-len(forecast)) + list(forecast)  # Align ARIMA forecast to last part
})
